# Notebook Title
 <details>
     <summary>&nbsp;<u>Details...</u></summary>

 **Notebook Version:** 1.0<br>
 **Python Version:** Python 3.6 (including Python 3.6 - AzureML)<br>
 **Required Packages**: kqlmagic, msticpy, pandas, pandas_bokeh, numpy, matplotlib, networkx, seaborn, datetime, ipywidgets, ipython, dnspython, ipwhois, folium, maxminddb_geolite2<br>
 **Platforms Supported**:
 - Azure Notebooks Free Compute
 - Azure Notebooks DSVM
 - OS Independent

 **Data Sources Required**:
 - Log Analytics/Azure Sentinel - Syslog, Secuirty Alerts, Auditd, Azure Network Analytics.
 - (Optional) - AlienVault OTX (requires account and API key)
 </details>

Notebook description....

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Notebook-Title" data-toc-modified-id="Notebook-Title-1">Notebook Title</a></span><ul class="toc-item"><li><span><a href="#Notebook-Initialization" data-toc-modified-id="Notebook-Initialization-1.1">Notebook Initialization</a></span><ul class="toc-item"><li><span><a href="#Get-WorkspaceId-and-Authenticate-to-Log-Analytics" data-toc-modified-id="Get-WorkspaceId-and-Authenticate-to-Log-Analytics-1.1.1">Get WorkspaceId and Authenticate to Log Analytics</a></span></li></ul></li><li><span><a href="#Example-query" data-toc-modified-id="Example-query-1.2">Example query</a></span></li></ul></li></ul></div>

---
### Notebook initialization
The next cell:
- Checks for the correct Python version
- Checks versions and optionally installs required packages
- Imports the required packages into the notebook
- Sets a number of configuration options.

This should complete without errors. If you encounter errors or warnings look at the following two notebooks:
- [TroubleShootingNotebooks](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/TroubleShootingNotebooks.ipynb)
- [ConfiguringNotebookEnvironment](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)

If you are running in the Azure Sentinel Notebooks environment (Azure Notebooks or Azure ML) you can run live versions of these notebooks:
- [Run TroubleShootingNotebooks](./TroubleShootingNotebooks.ipynb)
- [Run ConfiguringNotebookEnvironment](./ConfiguringNotebookEnvironment.ipynb)

You may also need to do some additional configuration to successfully use functions such as Threat Intelligence service lookup and Geo IP lookup. 
There are more details about this in the `ConfiguringNotebookEnvironment` notebook and in these documents:
- [msticpy configuration](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)
- [Threat intelligence provider configuration](https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html#configuration-file)


In [1]:
from pathlib import Path
from IPython.display import display, HTML

REQ_PYTHON_VER = "3.6"
REQ_MSTICPY_VER = "1.0.0"

display(HTML("<h3>Starting Notebook setup...</h3>"))
if Path("./utils/nb_check.py").is_file():
    from utils.nb_check import check_versions
    check_versions(REQ_PYTHON_VER, REQ_MSTICPY_VER)

# If not using Azure Notebooks, install msticpy with
# !pip install msticpy

extra_imports = []
# Usually there is no advantage to using nbinit to do your imports - just import them
# as normal. "init_notebook" imports a few standard packages (pandas, numpy, etc)
# and several common msticpy modules and classes.
# If you really want to use this mechanism the syntax is as follows:
# Each line is a string:
# - if just importing a module (e.g. "re"), just the name is enough
# - if importing an item from a module (e.g. from datetime import timedelta)
#   the string would be "datetime, delta"
# - if you want to import and alias something (e.g. import pandas as pd) us
#   "source_mod, , alias" (note you need the extra comma)
# - if you're importing an object from a module and want to alias it (e.g.
#   from datetime import timedelta as td - use "datetime, timedelta, td"
# extra_imports = [
#     "module.src [,target] [,alias",
#     "pandas, , pd",
#     "bokeh.plotting, show"
# ]

additional_packages = []
# specify the name of the package to install. It will not be installed if it
# is already. You can provide a package specification - e.g. pkg==version,
# as shown below
# additional_packages = ["seaborn", "another_pkg>=1.2.0"]

from msticpy.nbtools import nbinit
nbinit.init_notebook(
    namespace=globals(),
    extra_imports=extra_imports,
    additional_packages=additional_packages,
);


<a></a>[Contents](#toc)
### Get WorkspaceId and Authenticate to Log Analytics 

If you are using user/device authentication (the default), run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window).
  The device code will have been copied to the clipboard. 
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. 
- You should then be redirected to a user authentication page where you should 
  authenticate with a user account that has permission to query your Log Analytics workspace.

<details>
<summary> <u>Using an AppID and App Secret</u></summary>
Use the following syntax if you are authenticating using an Azure Active Directory AppId and Secret:<br>
<pre>
connect_str = "loganalytics://tenant(TENANT_ID).workspace(WORKSPACE_ID).clientid(client_id).clientsecret(client_secret)"
qry_prov.connect(connect_str)
</pre>
instead of<br>
<pre>
qry_prov.connect(ws_config)
</pre>

To find your Workspace Id go to
[Azure Sentinel Workspaces](https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces).
Look at the workspace properties to find the ID.
</details>

In [ ]:
# List Workspaces available
# WorkspaceConfig().list_workspaces()

# To use a specific workspace create a WorkspaceConfig using the
# workspace parameter
# ws_config = WorkspaceConfig(workspace='MyWorkspace')

In [ ]:
# See if we have an Azure Sentinel Workspace defined in our config file.
# If not, let the user specify Workspace and Tenant IDs

ws_config = WorkspaceConfig()
if not ws_config.config_loaded:
    ws_config.prompt_for_ws()
    
qry_prov = QueryProvider(data_environment="AzureSentinel")
print("done")


In [ ]:
# Authenticate to Azure Sentinel workspace
qry_prov.connect(ws_config)

In [ ]:
query_scope = nbwidgets.QueryTime(auto_display=True)

## Example query

In [ ]:
qry_prov.SecurityAlert.list_alerts(query_scope)